In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

# titanic data

Najpierw zaloguj się do https://www.kaggle.com/ i przejdź do wyzwania https://www.kaggle.com/c/titanic, aby pobrać 
 * train.csv i test.csv. 

Zapisz je w katalogu datasets/titanic.

In [6]:
import os
TITANIC_PATH = os.path.join("datasets")

In [7]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [8]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

* Dane są już podzielone na zestaw treningowy i zestaw testów. 
* Jednak dane testowe nie zawierają etykiet: Twoim celem jest wyszkolenie najlepszego modelu, który możesz wykorzystać w danych treningowych, następnie dokonanie swoich przewidywań na danych testowych i przesłanie ich do Kaggle, aby zobaczyć ostateczny wynik.

Rzućmy okiem na kilka pierwszych rzędów zestawu treningowego:

In [9]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The attributes have the following meaning:

* Survived: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* Pclass: passenger class.
* Name, Sex, Age: self-explanatory
* SibSp: how many siblings & spouses of the passenger aboard the Titanic.
* Parch: how many children & parents of the passenger aboard the Titanic.
* Ticket: ticket id
* Fare: price paid (in pounds)
* Cabin: passenger's cabin number
* Embarked: where the passenger embarked the Titanic
* Let's get more info to see how much data is missing:

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Atrybuty **Age**, **Cabin** oraz **Embarked** są czasami zerowe (mniej niż 891 wartości bez wartości null), szczególnie w przypadku **Cabin** (77% ma wartość zerową). Zignorujemy teraz **Cabin** i skupimy się na reszcie. Atrybut **Age** ma około 19% wartości pustych, więc będziemy musieli zdecydować, co z nimi zrobić. Zastąpienie wartości null medianą wieku wydaje się uzasadnione.

Atrybuty **Name** i **Ticket** mogą mieć pewną wartość, ale będą one nieco trudne do przekształcenia w użyteczne liczby. Na razie będziemy je ignorować.

Rzućmy okiem na atrybuty liczbowe:

In [11]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* Tylko 38% przeżyło: to wystarczająco blisko do 40%, więc **accuracy** będzie rozsądną miarą do oceny naszego modelu.

Sprawdźmy, czy etykiety przyjmują wartości 0 lub 1:

In [12]:
train_data['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

Nie zapomnij o etykietach:

In [13]:
y_train = train_data['Survived']

In [14]:
X_train = train_data.drop('Survived', axis=1)

Zamiana danych kategorycznych na liczby

In [15]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for column in X_train.columns:
    if X_train[column].dtype == 'object':  
        X_train[column] = label_encoder.fit_transform(X_train[column])

In [16]:
X_train.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,108,1,22.0,1,0,523,7.2500,147,2
1,2,1,190,0,38.0,1,0,596,71.2833,81,0
2,3,3,353,0,26.0,0,0,669,7.9250,147,2
3,4,1,272,0,35.0,1,0,49,53.1000,55,2
4,5,3,15,1,35.0,0,0,472,8.0500,147,2


In [17]:
X_train = X_train.drop(['Ticket', 'Name'], axis=1)

In [18]:
X_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,1,22.0,1,0,7.2500,147,2
1,2,1,0,38.0,1,0,71.2833,81,0
2,3,3,0,26.0,0,0,7.9250,147,2
3,4,1,0,35.0,1,0,53.1000,55,2
4,5,3,1,35.0,0,0,8.0500,147,2


Atrybut **Embarked** mówi nam, gdzie pasażer zaokrętował: C = Cherbourg, Q = Queenstown, S = Southampton.

Sprawdzam, gdzie są nulle do uzupełnienia


In [19]:
X_train.isnull().sum()

PassengerId      0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Cabin            0
Embarked         0
dtype: int64

Zapełniam nulle w wieku medianą

In [20]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

age_data_imputed = imputer.fit_transform(X_train[['Age']])

X_train['Age'] = age_data_imputed

In [21]:
X_train.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
dtype: int64

# Zad

Robimy StratifiedKFold i znajdujemy optymalne parametry dla

* SVM z jądrem rbf
* SVM z jądrem poly
* SVM liniowego
* Regresji logistycznej

In [22]:
data = X_train
target = y_train
target.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [23]:
np.isnan(data).sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( data, target, random_state=42)
seed=123

kfold = StratifiedKFold(n_splits=5,shuffle=True, random_state=seed)
param_grid = {'C': [0.2, 0.1, 0.09], 
              'kernel': ['linear']} 

print('XXX')
grid_1 = GridSearchCV(SVC(), param_grid, cv=kfold, return_train_score=True)

grid_1.fit(X_train, y_train)
grid_1.best_estimator_

XXX


SVC(C=0.2, kernel='linear')

In [171]:
param_grid = {'C': [  6],
              #'degree': [ 6, 7, 8],
            # 'coef0': [ 4, 5, 6],
             'kernel': ['poly']}

grid_2 = GridSearchCV(SVC(), param_grid, cv=kfold, return_train_score=True)

grid_2.fit(X_train, y_train)
grid_2.best_params_

{'C': 6, 'kernel': 'poly'}

In [26]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

grid_3= GridSearchCV(SVC(), param_grid, cv=kfold, return_train_score=True)

grid_3.fit(X_train, y_train)
grid_3.best_params_

{'C': 1, 'gamma': 0.001}

In [27]:
param_grid = {'C': [0.2, 0.25, 0.15], 
              'kernel': ['linear']} 

print('XXX')
grid_4 = GridSearchCV(SVC(), param_grid, cv=kfold, return_train_score=True)
grid_4.fit(X_train, y_train)
grid_4.best_params_

XXX


{'C': 0.2, 'kernel': 'linear'}

In [28]:
from sklearn.linear_model import LogisticRegression

param_grid = {
    'C': [0.1, 0.5, 1, 2, 3, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga', 'sag', 'lbfgs']
}

print('XXX')
grid_5 = GridSearchCV(LogisticRegression(), param_grid, cv=kfold, return_train_score=True)
grid_5.fit(X_train, y_train)
grid_5.best_params_

XXX


/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/sit

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/sit

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

{'C': 0.5, 'penalty': 'l1', 'solver': 'liblinear'}

In [170]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Dodanie transformatora StandardScaler
    ('classifier', xgb.XGBClassifier())  # Dodanie klasyfikatora
])



param_grid = {
    'scaler__with_mean': [True, False],  # Parametry dla StandardScaler
    'scaler__with_std': [True, False],
    'classifier__max_depth': [ 5, 6, 7],
    'classifier__learning_rate': [  0.001, 0.00005 ],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__subsample':[ 1.0, 0.9],
    
}

print('XXX')
grid_6 = GridSearchCV(pipeline, param_grid, cv=kfold, return_train_score=True)
grid_6.fit(X_train, y_train)
grid_6.best_params_

XXX


{'classifier__learning_rate': 0.001,
 'classifier__max_depth': 6,
 'classifier__n_estimators': 200,
 'classifier__subsample': 1.0,
 'scaler__with_mean': True,
 'scaler__with_std': True}

In [175]:
from sklearn.ensemble import AdaBoostClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Dodanie transformatora StandardScaler
    ('classifier', AdaBoostClassifier() )  # Dodanie klasyfikatora
])


param_grid = {
    'scaler__with_mean': [True, False],  # Parametry dla StandardScaler
    'scaler__with_std': [True, False],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__learning_rate': [0.1, 0.01, 0.001],
    'classifier__algorithm': ['SAMME', 'SAMME.R']
}

print('XXX')
grid_7 = GridSearchCV(pipeline, param_grid, cv=kfold, return_train_score=True)
grid_7.fit(X_train, y_train)
grid_7.best_params_

XXX


{'classifier__algorithm': 'SAMME.R',
 'classifier__learning_rate': 0.1,
 'classifier__n_estimators': 100,
 'scaler__with_mean': True,
 'scaler__with_std': True}

In [179]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Dodanie transformatora StandardScaler
    ('classifier', MLPClassifier() )  # Dodanie klasyfikatora
])

param_grid = {
    'scaler__with_mean': [True, False],  # Parametry dla StandardScaler
    'scaler__with_std': [True, False],
    'classifier__hidden_layer_sizes': [  (100, 50)],
    'classifier__activation': [ 'logistic'],
    'classifier__solver': ['adam'],
    'classifier__learning_rate': ['constant'],
    'classifier__alpha': [0.0001, 0.001, 0.01],
    'classifier__batch_size': [16, 32, 64],
    'classifier__learning_rate_init': [0.001, 0.01, 0.1],
    'classifier__max_iter': [100, 200, 300],
}

print('XXX')
grid_8 = GridSearchCV(pipeline, param_grid, cv=kfold, return_train_score=True)
grid_8.fit(X_train, y_train)
grid_8.best_params_

XXX


/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/mini

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/mini

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/mini

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/mini

...

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/mini

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/mini

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/mini

/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/student1/mini

{'classifier__activation': 'logistic',
 'classifier__alpha': 0.01,
 'classifier__batch_size': 64,
 'classifier__hidden_layer_sizes': (100, 50),
 'classifier__learning_rate': 'constant',
 'classifier__learning_rate_init': 0.01,
 'classifier__max_iter': 300,
 'classifier__solver': 'adam',
 'scaler__with_mean': False,
 'scaler__with_std': True}

In [208]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Dodanie transformatora StandardScaler
    ('classifier', RandomForestClassifier() )  # Dodanie klasyfikatora
])

param_grid = {
    'scaler__with_mean': [True, False],  # Parametry dla StandardScaler
    'scaler__with_std': [True, False],
    'classifier__n_estimators': [100],
    'classifier__max_depth': [None, 2, 5, 20],
    'classifier__min_samples_split': [4, 5, 10],
   # 'min_samples_leaf': [1, 2, 4],
  #  'max_features': ['sqrt', 'log2'],
   # 'classifier__bootstrap': [True, False]
}

print('XXX')
grid_9 = GridSearchCV(pipeline, param_grid, cv=kfold, return_train_score=True)
grid_9.fit(X_train, y_train)
grid_9.best_params_


XXX


{'classifier__max_depth': 5,
 'classifier__max_features': 'sqrt',
 'classifier__min_samples_split': 5,
 'classifier__n_estimators': 100,
 'scaler__with_mean': False}

In [210]:
from sklearn import  metrics


models = []
models.append(('SVM linear', grid_1.best_estimator_))
models.append(('SVM poly', grid_2.best_estimator_))
models.append(('SVM rbf', grid_3.best_estimator_))
models.append(('SVM Linear2', grid_4.best_estimator_))
models.append(('Logistic Regression', grid_5.best_estimator_))
models.append(('XGBoost', grid_6.best_estimator_))
models.append(('ADABoost', grid_7.best_estimator_))
models.append(('MLPClassifier', grid_8.best_estimator_))
models.append(('RandomForest', grid_9.best_estimator_))

print("XXX")

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test, model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))
    precision_score.append(metrics.precision_score(y_test, model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test, model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test, model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test, model.predict(X_test)))

XXX
SVM linear
precision_score: 0.7411764705882353
recall_score: 0.7078651685393258
f1_score: 0.7241379310344828
accuracy_score: 0.7847533632286996
SVM poly
precision_score: 0.85
recall_score: 0.19101123595505617
f1_score: 0.3119266055045872
accuracy_score: 0.6636771300448431
SVM rbf
precision_score: 0.6458333333333334
recall_score: 0.34831460674157305
f1_score: 0.45255474452554745
accuracy_score: 0.6636771300448431
SVM Linear2
precision_score: 0.7411764705882353
recall_score: 0.7078651685393258
f1_score: 0.7241379310344828
accuracy_score: 0.7847533632286996
Logistic Regression
precision_score: 0.75
recall_score: 0.7078651685393258
f1_score: 0.7283236994219654
accuracy_score: 0.7892376681614349
XGBoost
precision_score: 0.8181818181818182
recall_score: 0.7078651685393258
f1_score: 0.7590361445783134
accuracy_score: 0.820627802690583
ADABoost
precision_score: 0.7252747252747253
recall_score: 0.7415730337078652
f1_score: 0.7333333333333333
accuracy_score: 0.7847533632286996
MLPClassifier


In [211]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['SVM linear', 'SVM poly', 'SVM rbf', 'SVM Linear2', 'Logistic Regression', 'XGBoost', 'ADABoost', 'MLPClassifier', 'RandomForest'])
df

,Method,precision_score,recall_score,f1_score,accuracy_score
0,SVM linear,0.741176,0.707865,0.724138,0.784753
1,SVM poly,0.850000,0.191011,0.311927,0.663677
2,SVM rbf,0.645833,0.348315,0.452555,0.663677
3,SVM Linear2,0.741176,0.707865,0.724138,0.784753
4,Logistic Regression,0.750000,0.707865,0.728324,0.789238
5,XGBoost,0.818182,0.707865,0.759036,0.820628
6,ADABoost,0.725275,0.741573,0.733333,0.784753
7,MLPClassifier,0.842857,0.662921,0.742138,0.816143
8,RandomForest,0.840000,0.707865,0.768293,0.829596


In [141]:
test = test_data
test = test_data.drop(['Name', 'Ticket'], axis=1)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for column in test.columns:
    if test[column].dtype == 'object':  
        test[column] = label_encoder.fit_transform(test[column])

In [148]:
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           1
Cabin          0
Embarked       0
dtype: int64

In [149]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

age_data_imputed = imputer.fit_transform(test[['Age']])
fare_data_imputed = imputer.fit_transform(test[['Fare']])
test['Age'] = age_data_imputed
test['Fare'] = fare_data_imputed

In [150]:
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [212]:
# Uzyskanie predykcji na zestawie testowym
predictions = grid_9.predict(test)

# Konwersja predykcji na obiekt DataFrame
df_1 = pd.DataFrame(predictions, columns=['Survived'])
df_predictions = pd.DataFrame()
df_predictions['PassengerId'] = test['PassengerId']
df_predictions['Survived'] = df_1['Survived']

df_predictions.head()

# Zapisanie predykcji do pliku CSV
df_predictions.to_csv('predictions.csv', index=False)

In [213]:
df_predictions.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [214]:
df_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB
